In [1]:
!pip install selenium

  Using cached selenium-4.15.2-py3-none-any.whl (10.2 MB)
  Using cached trio-0.23.1-py3-none-any.whl (448 kB)
  Using cached trio_websocket-0.11.1-py3-none-any.whl (17 kB)
  Using cached sniffio-1.3.0-py3-none-any.whl (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)


  Using cached h11-0.14.0-py3-none-any.whl (58 kB)


In [2]:
!pip install selenium pandas

In [4]:
!pip install bs4

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1271 sha256=93d90ae3adcaa37c4eb83b34477c912666a19b9c1effc725606db7fb68e7bb03
  Stored in directory: c:\users\pooja\appdata\local\pip\cache\wheels\73\2b\cb\099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4


In [6]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

# Function to scrape the page using Selenium
def scrape_page(url):
    driver = webdriver.Chrome() 
    driver.get(url)
    
    # Wait for the page to load (you might need to adjust the time)
    driver.implicitly_wait(10)
    
    # Get the page source
    page_source = driver.page_source
    
    driver.quit()
    
    return page_source

# Function to extract paragraphs from HTML
def extract_paragraphs(html):
    soup = BeautifulSoup(html, 'html.parser')
    paragraphs = [p.get_text() for p in soup.find_all('p')]
    return paragraphs

# Scraping the Wikipedia page
url = "https://en.wikipedia.org/wiki/Nikola_Tesla"
page_source = scrape_page(url)

# Extracting paragraphs
paragraphs = extract_paragraphs(page_source)

In [33]:
paragraphs

['\n',
 'Nikola Tesla (/ˈtɛslə/; Serbian Cyrillic: Никола Тесла,[2] [nǐkola têsla];[a] 10 July [O.S. 28 June] 1856\xa0– 7 January 1943) was a Serbian-American[5][6] inventor, electrical engineer, mechanical engineer, and futurist. He is best-known for his contributions to the design of the modern alternating current (AC) electricity supply system.[7]\n',
 'Born and raised in the Austrian Empire, Tesla first studied engineering and physics in the 1870s without receiving a degree. He then gaining practical experience in the early 1880s working in telephony and at Continental Edison in the new electric power industry. In 1884 he emigrated to the United States, where he became a naturalized citizen. He worked for a short time at the Edison Machine Works in New York City before he struck out on his own. With the help of partners to finance and market his ideas, Tesla set up laboratories and companies in New York to develop a range of electrical and mechanical devices. His AC induction motor

In [34]:
#DATA CLEANING (DUPLICATES,NA VALUES)
# Create a Pandas DataFrame
df = pd.DataFrame({"Text": paragraphs})

# Basic data cleaning
df['Text'] = df['Text'].str.strip()

# Handling missing values
df.dropna(inplace=True)

# Removing duplicates
df.drop_duplicates(inplace=True)

# Saving cleaned data to a CSV file
df.to_csv('cleaned_data.csv', index=False)

In [35]:
data = pd.read_csv('cleaned_data.csv')

In [36]:
data.shape

(123, 1)

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    122 non-null    object
dtypes: object(1)
memory usage: 1.1+ KB


In [38]:
# Saving plain text to a text file
with open('plain_text.txt', 'w', encoding='utf-8') as file:
    file.write('\n'.join(paragraphs))

# Saving HTML text to a text file
with open('html_text.html', 'w', encoding='utf-8') as file:
    file.write(page_source)

In [45]:
# Extracting hyperlinks
soup = BeautifulSoup(page_source, 'html.parser')
links = soup.find_all('a', href=True)

# Ensure paragraphs and links have the same length
min_length = min(len(paragraphs), len(links))
paragraphs = paragraphs[:min_length]
links = links[:min_length]

# Create a DataFrame with 'Text' and 'Link' columns
links_df = pd.DataFrame({
    'Text': paragraphs,
    'Link': [link['href'] for link in links]
})

In [46]:
links_df

,Text,Link
0,\n,#bodyContent
1,Nikola Tesla (/ˈtɛslə/; Serbian Cyrillic: Нико...,/wiki/Main_Page
2,"Born and raised in the Austrian Empire, Tesla ...",/wiki/Wikipedia:Contents
3,Attempting to develop inventions he could pate...,/wiki/Portal:Current_events
4,"After Wardenclyffe, Tesla experimented with a ...",/wiki/Special:Random
...,...,...
118,Citations\n,https://gu.wikipedia.org/wiki/%E0%AA%A8%E0%AA%...
119,Books\n,https://ko.wikipedia.org/wiki/%EB%8B%88%EC%BD%...
120,Publications\n,https://ha.wikipedia.org/wiki/Nikola_Tesla
121,Journals\n,https://hy.wikipedia.org/wiki/%D5%86%D5%AB%D5%...


In [47]:
links_df.to_csv('links_data.csv', index=False)